In [26]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import engine
import pickle
import numpy as np

In [27]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
def get_release_date(trackID):
    print(trackID)
    song_release_date = sp.track(trackID)['album']['release_date']
    return song_release_date


In [3]:
df = pickle.load( open( "./song_list_v6.pkl", "rb"))

In [28]:
con = sqlite3.connect("song_list_curr.db")
df = pd.read_sql_query("SELECT * from songs", con)
con.close()

In [29]:
df = df.drop(['level_0'], axis=1)

In [30]:
df.head(2)

,index,songid,popularity,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,...,mode,speechiness,tempo,time_signature,track,valence,genres,genres_stripped,availability,release_date
0,0,5PS5dpaLogPzYU9hWiWyZb,0,0.001300,Karkkiautomaatti,0.487,157307.0,0.678,0.0551,9.0,...,1.0,0.0495,149.940,4.0,Tanssi vaan,0.870,"['finnish indie', 'suomi rock']","'finnish indie', 'suomi rock'",1.0,2006
1,1,41RpZW2lxAdnqDd2nMBzLQ,15,0.000045,Hudson Mohawke,0.662,138960.0,0.823,0.9520,4.0,...,0.0,0.0662,177.745,4.0,No One Could Ever,0.621,"['bass music', 'scottish electronic', 'scottis...","'bass music', 'scottish electronic', 'scottish...",1.0,2009-10-12


In [31]:
engine = create_engine('sqlite:///song_list_curr_na.db', echo=False)
df.to_sql('songs', con=engine)
engine.execute('SELECT * FROM songs;').fetchone()

(0, 0, '5PS5dpaLogPzYU9hWiWyZb', '0', 0.0013, 'Karkkiautomaatti', 0.487, 157307.0, 0.678, 0.0551, 9.0, 0.0846, -7.78, 1.0, 0.0495, 149.94, 4.0, 'Tanssi vaan', 0.87, "['finnish indie', 'suomi rock']", "'finnish indie', 'suomi rock'", 1.0, '2006')

In [32]:
for row_index in df.songid[df.index[df['release_date'] == 'NaN']]:
    songid = row_index
    #print(songid)
    date = get_release_date(songid)
    #print(date)
    try:
        engine.execute(f'UPDATE songs SET release_date = "{date}" WHERE songid = \"{songid}\"')
        authorize()
        #print(date)
    except:
        print(f"couldn't insert {date} into table")
        continue 

5us59QA2Rrb0y4FKYX7Tci
5slc0j5h3POUDcT3YhUYZg
5naGmbqRY9AlitnRgbw0uX
2YaKx3e7BFCWu4G0jtCfel
0NY7sR8WpyvO2axu8TWvdm
07Uhu501YGRK0WNPjcmPPa
5ELHJoMN4vMQPv9zMlKRjU
5CNhSMIM9N2n7v0ra6pmwv
47ciqrUH5cQQARvIQHbIUx
3t4FJ4JJUvChNM075zIpyZ
63PiAWEexq1C3R4vIDua9E
67HwOjFg2rJ6cntfbEJLrs
2fbTbfyhKAb3sKySsTeOy9
5xphO8cbL4bZ5k0zmahKV5
6favehcPh2nu5kDTJn4wcV
6g8qkrjWRvrJ72NQ5WDYsU
3NRFNfPbHQTVmnAWaEpZTF
3vM4sUIH4Plb26feersQHh
49a7X51JjTPeN8n2U62oxS
5yQ9aNDOmyMHzyA0Rnt5yr
3xca8zX9KWxOXLwMBYgS0z
2Y4fbPb8V9AgXAUat1IDjW
20iZ625orLce7Fn7IpFRn0
5Eb5nY60ZxvWcbBASx8Dhh
41o6dgtBlc9cXCJBjMG6iw
2t6Dyp6xlDq4ACc8VqX8FB
4Y5yjzs9FFw5qIgfZBd43I
6zkzdSPN5q5XHHlHfxu9EN
1nvVz58FrBl475j6cRGZbt
7paU74dQLr2K7sF5dW6vTt
6dJzRiQH35LP7M7S01e8cb
0hmq970mSIywv7wMFAZt2W
09lBsmuUqJ1twK1Jofwobf
7p9e1hTBxrHvdm1OEJ3R9a
3XP0PcHWE7f81FBHANwFRR
4KgGOKVvmw9FaRSxejAqhC
0XeoLi3EQVcXWOo8ySU4Vr
48ust1zjLKSkQzuwaDVZ0X
1KYHt955oMB9Rw4MSYDoZ7
369ZRKPX8UgrFDAKOcWk5R
1imN8gl6LNbhXTerceC4lX
7C2Th9WKovi6NyjGoqaPsK
7yO6MNYPCpuEbBB6JOn21s
42xBJZxqGgZ

In [33]:
df.songid[df.index[df['release_date'] == 'NaN']]

30157     5us59QA2Rrb0y4FKYX7Tci
33265     5slc0j5h3POUDcT3YhUYZg
34024     5naGmbqRY9AlitnRgbw0uX
36730     2YaKx3e7BFCWu4G0jtCfel
39424     0NY7sR8WpyvO2axu8TWvdm
                   ...          
565844    03Cr8fGw7ExcQZybJnsplR
567143    6o79O9qBP3OAkWOXajAQIS
567529    3dniCsjGVwye06F5bTy4Uo
570411    6UQEsGPUXpRIfXjerdwx4U
571469    6cHlBIY1NmdVVlHoSfwx0u
Name: songid, Length: 294, dtype: object